In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError
import tensorflow as tf

In [13]:
dataset=pd.read_csv('../data/weight_predict.csv',delimiter=',',encoding='utf8')
new_order=['나이','성별','신장','체중','활동량계수','BMR','에너지 섭취량(Kcal)','일일 칼로리 잉여/적자','체중변화']
dataset=dataset[new_order]
dataset

,나이,성별,신장,체중,활동량계수,BMR,에너지 섭취량(Kcal),일일 칼로리 잉여/적자,체중변화
0,56,1.0,174.928,103.600413,1.200,3102.3,3916.0,813.7,0.090718
1,46,2.0,162.000,75.024117,1.725,2275.5,3823.0,1547.5,1.088621
2,32,2.0,161.984,64.772938,1.200,2119.4,2785.4,666.0,0.635029
3,25,2.0,161.968,65.997636,1.200,2181.3,2587.3,406.0,0.362874
4,38,1.0,175.008,70.533556,1.375,2463.8,3312.8,849.0,0.907184
...,...,...,...,...,...,...,...,...,...
95,59,1.0,175.008,82.508385,1.375,2622.8,3656.2,1033.4,0.272155
96,56,2.0,162.000,52.843468,1.550,1736.5,2756.5,1020.0,-4.047442
97,58,2.0,162.016,73.799418,1.375,2188.6,3344.3,1155.8,1.632931
98,45,1.0,175.024,81.555842,1.550,2671.9,4000.0,1328.1,0.907184


In [14]:
# BMR 계산 함수
def calculate_bmr(dataset):
    if dataset["성별"] == 1.0:  # 남성
        return 10 * dataset["체중"] + 6.25 * dataset["신장"] - 5 * dataset["나이"] + 5
    elif dataset["성별"] == 2.0:  # 여성
        return 10 * dataset["체중"] + 6.25 * dataset["신장"] - 5 * dataset["나이"] - 161

# BMR 열 추가
dataset["BMR"] = dataset.apply(calculate_bmr, axis=1)

dataset['에너지 섭취량(Kcal)'] =dataset['에너지 섭취량(Kcal)']*0.453592*dataset['활동량계수']
dataset['일일 칼로리 잉여/적자'] =dataset['에너지 섭취량(Kcal)']-dataset["BMR"]*dataset['활동량계수']
dataset

,나이,성별,신장,체중,활동량계수,BMR,에너지 섭취량(Kcal),일일 칼로리 잉여/적자,체중변화
0,56,1.0,174.928,103.600413,1.200,1854.304128,2131.519526,-93.645427,0.090718
1,46,2.0,162.000,75.024117,1.725,1371.741168,2991.291823,625.038308,1.088621
2,32,2.0,161.984,64.772938,1.200,1339.129376,1516.122188,-90.833063,0.635029
3,25,2.0,161.968,65.997636,1.200,1386.276360,1408.294298,-255.237334,0.362874
4,38,1.0,175.008,70.533556,1.375,1614.135560,2066.156919,-153.279476,0.907184
...,...,...,...,...,...,...,...,...,...
95,59,1.0,175.008,82.508385,1.375,1628.883848,2280.331722,40.616431,0.272155
96,56,2.0,162.000,52.843468,1.550,1099.934680,1938.005839,233.107085,-4.047442
97,58,2.0,162.016,73.799418,1.375,1299.594184,2085.803123,298.861120,1.632931
98,45,1.0,175.024,81.555842,1.550,1689.458416,2812.270400,193.609855,0.907184
